In [ ]:
#Import necessary libraries and change working directory.

import pandas as pd
import os
import requests
import datetime

working_directory = '/Users/maxwell/Desktop/Code/SALMONQUEST/SalmonData'
os.chdir(working_directory)
currpath = os.getcwd()
directory = os.listdir(currpath)

In [ ]:
"""Data Import"""

In [ ]:
"""
The Columbia River has 9 salmon counting locations at increasingly-inland points along the river.
For each of these dams, I will extract the daily count CSV file for each year from 2006 to 2022.
I will create 10 CSV files.
There will be 9 long-format CSV files for each dam, each including the entire year range.
There will also be one master CSV file. This will be the total collection of data.

"""

#Each of the dams' full names and their abbreviation used in the HTTP request.
dam_names = {
    'Bonneville': 'BON',
    'The Dalles': 'TDA',
    'John Day': 'JDA',
    'McNary': 'MCN',
    'Priest Rapids': 'PRD',
    'Wanapum': 'WAN',
    'Rock Island': 'RIS',
    'Rocky Beach': 'RRH',
    'Wells': 'WEL'
}

#Check if data already exists in working directory. If so, an exception is thrown.
#The CSV files are opened and manipulated by appending, so if this block is run more than once,
#there will be redundant data.
if os.path.exists(f'{currpath}/master.csv') is True:
    raise NameError('Data is already stored locally')

#Create CSVs
master_file = open('master.csv', 'at')
for key in dam_names:
    dam = dam_names[key]
    csv_file = open(f'{dam}_counts.csv', 'at')
    
#In the following block, the csv files of each year for each dam are appended to the dam's csv file.
    for year in range(2006, 2023):
        year = str(year)
        
#The CSV file is located at this url. I reformat the url based on the year and the dam abbreviation.
        url = f'https://www.cbr.washington.edu/dart/cs/php/rpt/adult_daily.php?sc=1&outputFormat=csv&year={year}&proj={dam}&span=no&startdate=1%2F1&enddate=12%2F31&run=&syear={year}&eyear={year}'
        req = requests.get(url)
        
#If there is no data for that year, the url will redirect to an error page.
#If this happens, the year is skipped.
        if url != req.url:
            continue
        url_content = req.text
        
#There is a 'Notes' section at the end of each file. Here, I slice out that portion of the string.
        end_of_data = url_content.find('Notes:')
        url_content = url_content[:end_of_data]
        csv_file.write(url_content)
        master_file.write(url_content)
    csv_file.close()
    print(f'{dam} done.')
print('All files filled.')
master_file.close()
print('Master file filled.')

#Create pandas database for each dam csv file, and master CSV file.
BON = pd.read_csv('BON_counts.csv')
TDA = pd.read_csv('TDA_counts.csv')
JDA = pd.read_csv('JDA_counts.csv')
MCN = pd.read_csv('MCN_counts.csv')
PRD = pd.read_csv('PRD_counts.csv')
WAN = pd.read_csv('WAN_counts.csv')
RIS = pd.read_csv('RIS_counts.csv')
RRH = pd.read_csv('RRH_counts.csv')
WEL = pd.read_csv('WEL_counts.csv')
MASTER = pd.read_csv('master.csv')

In [ ]:
"""Data Cleaning"""

In [ ]:
#Explore Data

MAS
MAS.shape
MAS.info
MAS.columns
MAS.isnull().sum()

In [ ]:
#Reformat 1: Rename columns
MAS = MAS.rename(
    columns={
        'Project': 'Location',
        'Chin': 'Chinook',
        'Sock': 'Sockeye',
    })

#Reformat 2: Remove all fish types that aren't adult salmon.
#Only Chinook, Sockeye, and Coho are the ones I would eat for dinner !
column_names = ['Location', 'Date', 'Chinook','Sockeye','Coho']
MAS = MAS[column_names]

#Reformat 3: Change null salmon counts to zeroes.
MAS = MAS.fillna(0)

#Reformat 4: Order entries by Date; then by Location.
MAS = MAS.sort_values(by=['Date', 'Location'], ignore_index=True)

#Reformat 5: Add total salmon count column.
MAS['Total Salmon'] = (MAS['Chinook']+MAS['Sockeye']+MAS['Coho'])

#Reformat 6: Include data from Apr to Oct only.


#Reformat 7: Take average of all salmon count data points for each day of the year.
ds2 = ds1.groupby('Date', as_index=False)['Total Salmon'].mean()
ds2['Total Salmon'] = ds2['Total Salmon'].astype(int)

In [ ]:
"""Data Analysis"""

In [ ]:
#Import simple dataset into Python
os.chdir('./Code/SALMONQUEST/SalmonData')

dataanalysispath = 'data_simple.csv'
#Define simple dataset as ds.
ds = pd.DataFrame(pd.read_csv(dataanalysispath))

In [ ]:
#QUESTION:
#What are the top 20 salmon totals and their corresponding dates?
top_20_set = ds.sort_values(by='Total Salmon', ascending=False, ignore_index = True).head(20)

#Top 20 salmon totals
top_20_nums = list(top_20_set['Total Salmon'])

#Top salmon count
high_num = max(top_20_nums)

#For each salmon count, what is it's percent value compared to the top salmon count?
percentages = []
for i in top_20_nums:
    percentages.append((i/high_num)*100)
    print(percentages[len(percentages)-1])

In [ ]:
"""
Conclusions:
Top 12 days are in July. Top day is July 8th.
Compared to the day with the most salmon, only the top 7 days have 75% or more as many fish,
and only the top 18 days have 50% or more as many fish. 
"""

In [ ]:
#Find Peaks and Quantify Change each day
ds['Net Change'] = ds['Total Salmon'].diff()
top_20_change = ds.sort_values(by='Net Change', ascending=False, ignore_index=True)
top_20_change.head(20)